<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [3]:
!pip3 install beautifulsoup4
!pip3 install requests

In [4]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [2]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# URL for the Falcon 9 and Falcon Heavy Launches Wikipedia page (static version)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request to retrieve the page content
response = requests.get(static_url)

# Check if the request was successful (HTTP status code 200)
print(response.status_code)


200


Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup

# Create a BeautifulSoup object from the HTML response
soup = BeautifulSoup(response.content, "html.parser")

# Check the first 500 characters to verify the content
print(soup.prettify()[:500])


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
# Print the title of the page to verify if the BeautifulSoup object was created properly
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Find all tables in the HTML page
# Step 1: Find all tables on the Wikipedia page
tables = soup.find_all("table", class_="wikitable")

# Step 2: Initialize an empty list to store column names
column_names = []

# Step 3: Iterate over the tables to extract column names
for table in tables:
    headers = table.find_all("th")  # Find all table headers (<th>)
    for header in headers:
        column_name = header.get_text(strip=True)  # Get the text content of the header
        if column_name:  # Only add non-empty column names
            column_names.append(column_name)

# Print out the extracted column names
print(column_names)




['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7', 'Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '8', '9', '10', '11', '12', '13', 'Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20', 'Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '21', '22', '23', '24', '25', '26', '27', '28', 'N/A[e]', 'Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '29', '30', '31', '32', '33', '34',

Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
# Step 1: Get the first launch table (assuming it's the third table in the list)
first_launch_table = tables[2]

# Step 2: Print the content of the first launch table
print(first_launch_table.prettify())  # .prettify() makes the HTML more readable


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-2">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-2">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [13]:

# Assuming `first_launch_table` is the correct table we are working with
header_cells = first_launch_table.find_all('th')

# Initialize an empty list to store non-empty column names
column_names = []
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name


# Iterate through each <th> element in the header row and extract the column name
for th in header_cells:
    # Extract column name using the provided helper function
    column_name = extract_column_from_header(th)
    
    # Check if the column name is non-empty and append it to the list
    if column_name and len(column_name) > 0:
        column_names.append(column_name)



Check the extracted column names


In [14]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [16]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

In [20]:
# Initialize variables
extracted_row = 0
launch_dict = []

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get each table row
    for rows in table.find_all("tr"):
        # Check if first table heading corresponds to launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get table elements (td cells)
        row = rows.find_all('td')
        
        # If it is a valid row (flight number found)
        if flag:
            extracted_row += 1
            
            # Flight Number value
            launch_dict.append({
                "Flight No.": flight_number
            })
            
            # Date and Time values
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            
            # Append date and time to the dictionary
            launch_dict[-1]["Date"] = date
            launch_dict[-1]["Time"] = time
            
            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict[-1]["Version Booster"] = bv
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict[-1]["Launch Site"] = launch_site
            
            # Payload
            payload = row[3].a.string
            launch_dict[-1]["Payload"] = payload
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict[-1]["Payload Mass"] = payload_mass
            
            # Orbit
            orbit = row[5].a.string
            launch_dict[-1]["Orbit"] = orbit
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict[-1]["Customer"] = customer
            
            # Launch Outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict[-1]["Launch Outcome"] = launch_outcome
            
            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict[-1]["Booster Landing"] = booster_landing

# Print the launch_dict to see the result
print(launch_dict)


[{'Flight No.': '1', 'Date': '4 June 2010', 'Time': '18:45', 'Version Booster': 'F9 v1.07B0003.18', 'Launch Site': 'CCAFS', 'Payload': 'Dragon Spacecraft Qualification Unit', 'Payload Mass': 0, 'Orbit': 'LEO', 'Customer': 'SpaceX', 'Launch Outcome': 'Success\n', 'Booster Landing': 'Failure'}, {'Flight No.': '2', 'Date': '8 December 2010', 'Time': '15:43', 'Version Booster': 'F9 v1.07B0004.18', 'Launch Site': 'CCAFS', 'Payload': 'Dragon', 'Payload Mass': 0, 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch Outcome': 'Success', 'Booster Landing': 'Failure'}, {'Flight No.': '3', 'Date': '22 May 2012', 'Time': '07:44', 'Version Booster': 'F9 v1.07B0005.18', 'Launch Site': 'CCAFS', 'Payload': 'Dragon', 'Payload Mass': '525 kg', 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch Outcome': 'Success', 'Booster Landing': 'No attempt\n'}, {'Flight No.': '4', 'Date': '8 October 2012', 'Time': '00:35', 'Version Booster': 'F9 v1.07B0006.18', 'Launch Site': 'CCAFS', 'Payload': 'SpaceX CRS-1', 'Payload Mass': '4,

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [18]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict() })

TypeError: 'list' object is not callable

In [22]:
import pandas as pd

# Assuming launch_dict is a list of dictionaries (each dictionary represents a launch record)
# You can directly create a DataFrame from the list of dictionaries

df = pd.DataFrame(launch_dict)

# Print the DataFrame
print(df)


    Flight No.             Date   Time   Version Booster Launch Site  \
0            1      4 June 2010  18:45  F9 v1.07B0003.18       CCAFS   
1            2  8 December 2010  15:43  F9 v1.07B0004.18       CCAFS   
2            3      22 May 2012  07:44  F9 v1.07B0005.18       CCAFS   
3            4   8 October 2012  00:35  F9 v1.07B0006.18       CCAFS   
4            5     1 March 2013  15:10  F9 v1.07B0007.18       CCAFS   
..         ...              ...    ...               ...         ...   
116        117       9 May 2021  06:42  F9 B5B1051.10657       CCSFS   
117        118      15 May 2021  22:56   F9 B5B1058.8660         KSC   
118        119      26 May 2021  18:59   F9 B5B1063.2665       CCSFS   
119        120      3 June 2021  17:29   F9 B5B1067.1668         KSC   
120        121      6 June 2021  04:26             F9 B5       CCSFS   

                                  Payload Payload Mass Orbit   Customer  \
0    Dragon Spacecraft Qualification Unit            0   LEO

In [24]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Step 1: Fetch the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Step 3: Find all tables with class "wikitable"
tables = soup.find_all("table", class_="wikitable")

# Step 4: Initialize counters for Falcon 9 and Falcon 1 launches
falcon_9_count = 0
falcon_1_count = 0
date_cutoff = datetime(2021, 6, 9)  # June 9, 2021 cutoff

# Step 5: Iterate through all rows in the first table (assuming it's the relevant one)
for table in tables:
    rows = table.find_all("tr")  # Find all rows in the table
    for row in rows:
        columns = row.find_all("td")  # Find all columns in the row
        if len(columns) > 1:  # Check if it's a valid data row (not a header)
            rocket_type = columns[1].text.strip()  # Rocket info is usually in the second column
            launch_date_text = columns[0].text.strip()  # Date is usually in the first column
            
            # Parse the date from the launch date string
            try:
                launch_date = datetime.strptime(launch_date_text, "%d %B %Y")
            except ValueError:
                continue  # Skip rows with invalid date formats
            
            # Only consider launches up until June 9, 2021
            if launch_date <= date_cutoff:
                if "Falcon 9" in rocket_type:
                    falcon_9_count += 1  # Increment Falcon 9 count
                elif "Falcon 1" in rocket_type:
                    falcon_1_count += 1  # Increment Falcon 1 count

# Step 6: Calculate Falcon 9 launches after removing Falcon 1 launches
falcon_9_after_removal = falcon_9_count - falcon_1_count

# Step 7: Print the number of Falcon 9 launches after removing Falcon 1 launches
print(f"Number of Falcon 9 launches after removing Falcon 1 launches until June 9, 2021: {falcon_9_after_removal}")


Number of Falcon 9 launches after removing Falcon 1 launches until June 9, 2021: 0


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
